In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('bgan')):
    os.mkdir('saved_models_mnist_{}'.format('bgan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('bgan'), mode='w')
import torch.utils.data as Data

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K

import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import numpy as np
import os
class BGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.boundary_loss, optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def boundary_loss(self, y_true, y_pred):
        """
        Boundary seeking loss.
        Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/
        """
        return 0.5 * K.mean((K.log(y_pred) - K.log(1 - y_pred))**2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test,y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))
                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    bgan = BGAN()
    bgan.train(epochs=30, batch_size=64, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.543107, acc.: 50.78%] [G loss: 0.298934]
epoch:0 step:2 [D loss: 0.485043, acc.: 59.38%] [G loss: 0.109027]
epoch:0 step:3 [D loss: 0.366065, acc.: 75.00%] [G loss: 0.150807]
epoch:0 step:4 [D loss: 0.256559, acc.: 93.75%] [G loss: 0.377074]
epoch:0 step:5 [D loss: 0.226749, acc.: 97.66%] [G loss: 0.650972]
epoch:0 step:6 [D loss: 0.168205, acc.: 100.00%] [G loss: 0.883893]
epoch:0 step:7 [D loss: 0.162598, acc.: 100.00%] [G loss: 1.106005]
epoch:0 step:8 [D loss: 0.117471, acc.: 100.00%] [G loss: 1.324349]
epoch:0 step:9 [D loss: 0.132987, acc.: 100.00%] [G loss: 1.316670]
epoch:0 step:10 [D loss: 0.113404, acc.: 100.00%] [G loss: 1.588310]
epoch:0 step:11 [D loss: 0.110259, acc.: 100.00%] [G loss: 1.651841]
epoch:0 step:12 [D loss: 0.101780, acc.: 100.00%] [G loss: 1.903466]
epoch:0 step:13 [D loss: 0.088110, acc.: 100.00%] [G loss: 2.007510]
epoch:0 step:14 [D loss: 0.088028, acc.: 100.00%] [G loss: 2.278149]
epoch:0 step:15 [D loss: 0.081257, acc.: 100.00%

epoch:0 step:121 [D loss: 0.057464, acc.: 100.00%] [G loss: 9.109513]
epoch:0 step:122 [D loss: 0.353837, acc.: 83.59%] [G loss: 9.608680]
epoch:0 step:123 [D loss: 0.178020, acc.: 92.97%] [G loss: 7.406180]
epoch:0 step:124 [D loss: 0.147613, acc.: 94.53%] [G loss: 8.493693]
epoch:0 step:125 [D loss: 0.413089, acc.: 82.81%] [G loss: 9.821875]
epoch:0 step:126 [D loss: 0.723798, acc.: 73.44%] [G loss: 6.927329]
epoch:0 step:127 [D loss: 0.198574, acc.: 90.62%] [G loss: 9.713606]
epoch:0 step:128 [D loss: 0.097447, acc.: 96.09%] [G loss: 12.247216]
epoch:0 step:129 [D loss: 0.143822, acc.: 96.09%] [G loss: 11.634010]
epoch:0 step:130 [D loss: 0.275716, acc.: 91.41%] [G loss: 7.175950]
epoch:0 step:131 [D loss: 0.081023, acc.: 97.66%] [G loss: 9.156990]
epoch:0 step:132 [D loss: 0.074912, acc.: 97.66%] [G loss: 9.167073]
epoch:0 step:133 [D loss: 0.147457, acc.: 96.09%] [G loss: 7.600654]
epoch:0 step:134 [D loss: 0.134759, acc.: 96.09%] [G loss: 8.189727]
epoch:0 step:135 [D loss: 0.176

...

epoch:29 step:27578 [D loss: 0.502536, acc.: 72.66%] [G loss: 0.838890]
epoch:29 step:27579 [D loss: 0.479556, acc.: 75.00%] [G loss: 0.748869]
epoch:29 step:27580 [D loss: 0.614995, acc.: 65.62%] [G loss: 0.717180]
epoch:29 step:27581 [D loss: 0.545355, acc.: 69.53%] [G loss: 0.720265]
epoch:29 step:27582 [D loss: 0.539556, acc.: 71.09%] [G loss: 0.730023]
epoch:29 step:27583 [D loss: 0.561036, acc.: 67.19%] [G loss: 0.681880]
epoch:29 step:27584 [D loss: 0.591899, acc.: 67.97%] [G loss: 0.612876]
epoch:29 step:27585 [D loss: 0.563326, acc.: 69.53%] [G loss: 0.649026]
epoch:29 step:27586 [D loss: 0.539125, acc.: 74.22%] [G loss: 0.527765]
epoch:29 step:27587 [D loss: 0.459645, acc.: 77.34%] [G loss: 0.620619]
epoch:29 step:27588 [D loss: 0.497476, acc.: 74.22%] [G loss: 0.911078]
epoch:29 step:27589 [D loss: 0.483235, acc.: 75.00%] [G loss: 0.894461]
epoch:29 step:27590 [D loss: 0.579307, acc.: 70.31%] [G loss: 0.822914]
epoch:29 step:27591 [D loss: 0.606728, acc.: 63.28%] [G loss: 0.

epoch:29 step:27694 [D loss: 0.555103, acc.: 75.00%] [G loss: 0.720865]
epoch:29 step:27695 [D loss: 0.531176, acc.: 73.44%] [G loss: 0.850337]
epoch:29 step:27696 [D loss: 0.458891, acc.: 75.00%] [G loss: 0.847674]
epoch:29 step:27697 [D loss: 0.508804, acc.: 72.66%] [G loss: 0.720579]
epoch:29 step:27698 [D loss: 0.556700, acc.: 72.66%] [G loss: 0.709861]
epoch:29 step:27699 [D loss: 0.497006, acc.: 73.44%] [G loss: 0.801049]
epoch:29 step:27700 [D loss: 0.567340, acc.: 69.53%] [G loss: 0.799213]
epoch:29 step:27701 [D loss: 0.654725, acc.: 64.84%] [G loss: 0.662664]
epoch:29 step:27702 [D loss: 0.556430, acc.: 68.75%] [G loss: 0.545465]
epoch:29 step:27703 [D loss: 0.565407, acc.: 64.84%] [G loss: 0.619898]
epoch:29 step:27704 [D loss: 0.535926, acc.: 71.09%] [G loss: 0.836400]
epoch:29 step:27705 [D loss: 0.550753, acc.: 71.09%] [G loss: 0.684346]
epoch:29 step:27706 [D loss: 0.468310, acc.: 77.34%] [G loss: 0.748299]
epoch:29 step:27707 [D loss: 0.493683, acc.: 74.22%] [G loss: 0.

epoch:29 step:27808 [D loss: 0.434281, acc.: 79.69%] [G loss: 0.806829]
epoch:29 step:27809 [D loss: 0.632066, acc.: 65.62%] [G loss: 0.623955]
epoch:29 step:27810 [D loss: 0.523422, acc.: 71.88%] [G loss: 0.703677]
epoch:29 step:27811 [D loss: 0.525521, acc.: 73.44%] [G loss: 0.595292]
epoch:29 step:27812 [D loss: 0.475456, acc.: 74.22%] [G loss: 0.839146]
epoch:29 step:27813 [D loss: 0.515741, acc.: 73.44%] [G loss: 1.028958]
epoch:29 step:27814 [D loss: 0.499148, acc.: 78.12%] [G loss: 0.874975]
epoch:29 step:27815 [D loss: 0.474721, acc.: 76.56%] [G loss: 1.052213]
epoch:29 step:27816 [D loss: 0.526973, acc.: 70.31%] [G loss: 0.828840]
epoch:29 step:27817 [D loss: 0.577788, acc.: 65.62%] [G loss: 0.725522]
epoch:29 step:27818 [D loss: 0.521574, acc.: 71.09%] [G loss: 0.816354]
epoch:29 step:27819 [D loss: 0.601209, acc.: 68.75%] [G loss: 0.696066]
epoch:29 step:27820 [D loss: 0.449336, acc.: 78.12%] [G loss: 0.813722]
epoch:29 step:27821 [D loss: 0.367333, acc.: 82.03%] [G loss: 1.

epoch:29 step:27924 [D loss: 0.467213, acc.: 75.78%] [G loss: 1.030152]
epoch:29 step:27925 [D loss: 0.567319, acc.: 67.97%] [G loss: 0.867433]
epoch:29 step:27926 [D loss: 0.466251, acc.: 78.12%] [G loss: 0.945882]
epoch:29 step:27927 [D loss: 0.548985, acc.: 68.75%] [G loss: 0.612002]
epoch:29 step:27928 [D loss: 0.483416, acc.: 78.12%] [G loss: 0.801453]
epoch:29 step:27929 [D loss: 0.569207, acc.: 70.31%] [G loss: 0.768272]
epoch:29 step:27930 [D loss: 0.553879, acc.: 69.53%] [G loss: 0.646609]
epoch:29 step:27931 [D loss: 0.531556, acc.: 75.00%] [G loss: 0.766566]
epoch:29 step:27932 [D loss: 0.496399, acc.: 74.22%] [G loss: 0.869784]
epoch:29 step:27933 [D loss: 0.496769, acc.: 73.44%] [G loss: 0.858002]
epoch:29 step:27934 [D loss: 0.550760, acc.: 66.41%] [G loss: 0.673447]
epoch:29 step:27935 [D loss: 0.560064, acc.: 68.75%] [G loss: 0.724171]
epoch:29 step:27936 [D loss: 0.537920, acc.: 71.88%] [G loss: 0.674915]
epoch:29 step:27937 [D loss: 0.548341, acc.: 71.09%] [G loss: 0.

epoch:29 step:28040 [D loss: 0.637902, acc.: 65.62%] [G loss: 0.510232]
epoch:29 step:28041 [D loss: 0.534511, acc.: 71.88%] [G loss: 0.545449]
epoch:29 step:28042 [D loss: 0.589068, acc.: 67.19%] [G loss: 0.633234]
epoch:29 step:28043 [D loss: 0.441262, acc.: 78.12%] [G loss: 0.884285]
epoch:29 step:28044 [D loss: 0.473238, acc.: 78.12%] [G loss: 0.881273]
epoch:29 step:28045 [D loss: 0.508110, acc.: 75.78%] [G loss: 0.593831]
epoch:29 step:28046 [D loss: 0.582947, acc.: 70.31%] [G loss: 0.600547]
epoch:29 step:28047 [D loss: 0.597605, acc.: 66.41%] [G loss: 0.606424]
epoch:29 step:28048 [D loss: 0.484109, acc.: 76.56%] [G loss: 0.773792]
epoch:29 step:28049 [D loss: 0.532281, acc.: 72.66%] [G loss: 0.756905]
epoch:29 step:28050 [D loss: 0.593957, acc.: 65.62%] [G loss: 0.609223]
epoch:29 step:28051 [D loss: 0.499637, acc.: 72.66%] [G loss: 0.682655]
epoch:29 step:28052 [D loss: 0.539310, acc.: 69.53%] [G loss: 0.642323]
epoch:29 step:28053 [D loss: 0.647798, acc.: 60.16%] [G loss: 0.